In [1]:
import pymks
# %matplotlib inline
# %load_ext autoreload
# %autoreload 2


In [2]:
from pymks import MKSStructureAnalysis
from pymks.bases import LegendreBasis
from pymks.datasets import CahnHilliardSimulation
import numpy as np
import cPickle as pickle


In [3]:
try:
    X = pickle.load(open('test_stats_data.pkl', 'rb'))
except:
    sim = CahnHilliardSimulation(gamma=0.4) # 0.4
    np.random.seed(12) # 0, 1
    means = (0.0008 * np.random.random(10) - 0.0004)
    print means
    X0 = np.concatenate([1e-5 * np.random.randn(1, 95, 95) - m for m in means])

    X = X0.copy()
    X_1000 = []
    coeffs = []
    coeffs_1000 = []
    # savemat('data_more_more/data_step_0000', {'data': X})
    for ii in range(4000):
        sim.run(X)
        X = sim.response
        if (ii + 1) % 50 == 0:
            X_1000.append(X)
    X = np.concatenate(X_1000)
    print X.shape
    X = X.reshape((80, 10, 95, 95))
    X = np.transpose(X, [1, 0, 2, 3])
    X = X.reshape((800, 95, 95))
    pickle.dump(X, open('test_stats_data.pkl', 'wb'))
print X.shape

(800, 95, 95)


In [4]:
corr=[(1, 1), (3, 3)] 

In [5]:
from pymks.stats import correlate
l_basis = LegendreBasis(n_states=10, domain=[-1.1, 1.1])

X_corr = correlate(X, l_basis, periodic_axes=[0, 1], correlations=corr)


In [6]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation


fig, (ax1, ax2) = plt.subplots(2,1)

imgs = []
for i in range(len(X)):
    im0 = ax1.imshow(X_corr[i, ..., 0])
    ax1.set_title('Stats', fontsize=20)
#     ax1.colorbar()
    im1 = ax2.imshow(X[i])
    ax2.set_title('Structure', fontsize=20)
#     ax2.colorbar()
    imgs.append([im0, im1])

im_ani1 = animation.ArtistAnimation(fig, imgs, interval=50, repeat_delay=300,
    blit=True)


plt.show()